In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_bokeh as pb
import numpy as np
import random
import datetime

In [31]:
filename = 'potential.202101.pkl'

In [32]:
databook = pd.read_pickle("Data/History/%s" % filename, compression='bz2')

### 将渠道补全

In [33]:
databook['客户来源'] = databook['客户来源_原始'].fillna('展厅')

In [34]:
databook.head(2)

,日期,大区名称,小区名称,经销商代码,分组名称,客户来源_原始,潜客量,客户来源
0,2021-01-01,北部区,北京1区,SA11003,A3 SpB,线上,1,线上
1,2021-01-01,北部区,北京1区,SA11003,A4 AV,展厅,1,展厅


### 分两种部分，1. 做为标准的部分 2. 待补充的部分

In [35]:
dataSTD = databook[databook['分组名称'].notna()]
data2b = databook[databook['分组名称'].isna()]

In [125]:
dataSTD['dealer汇总'] = dataSTD.groupby(['日期', '经销商代码', '客户来源'])['潜客量'].transform(sum)

/home/orin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
dataSTD['车型占比'] = dataSTD['潜客量']/dataSTD['dealer汇总']

/home/orin/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### 先检查是否有缺失，运行`checkMISS`函数，会返回结果。
### 当前只计算 **日期+经销商** 和 **日期+大区名称** 层级

In [118]:
def checkMISS():
    a = dataSTD[['日期', '大区名称', '经销商代码', '客户来源']].drop_duplicates()
    a['std'] = True
    b = data2b[['日期', '大区名称', '经销商代码', '客户来源']].drop_duplicates()
    b['2b'] = True
    c = a.merge(b, on=['日期', '大区名称', '经销商代码', '客户来源'], how='outer')
    d = a[['日期', '大区名称', '客户来源', 'std']].merge(b[['日期', '大区名称', '客户来源', '2b']], on=['日期', '大区名称', '客户来源'], how='outer')
    print('日期+经销商代码', not c['std'].isna().any())
    print('日期+大区名称', not d['std'].isna().any())

In [120]:
checkMISS()

日期+经销商代码 False
日期+大区名称 True


In [22]:
dataSTD.groupby(['日期', '大区名称', '小区名称', '经销商代码', '分组名称', '客户来源'])['潜客量'].sum().reset_index()

,日期,大区名称,小区名称,经销商代码,分组名称,客户来源,潜客量
0,2021-01-01,东北区,吉林小区,SA15016,A4 AR,展厅,1
1,2021-01-01,东北区,吉林小区,SA15016,A4L,展厅,1
2,2021-01-01,东北区,吉林小区,SA15016,A4L,线上,1
3,2021-01-01,东北区,吉林小区,SA15016,A5 SpB,展厅,3
4,2021-01-01,东北区,吉林小区,SA15016,A6 AV,展厅,19
5,2021-01-01,东北区,吉林小区,SA15016,A6L,展厅,2
6,2021-01-01,东北区,吉林小区,SA15016,A6L,线上,1
7,2021-01-01,东北区,吉林小区,SA15016,A8L,展厅,1
8,2021-01-01,东北区,吉林小区,SA15016,Q5L,展厅,2
9,2021-01-01,东北区,吉林小区,SA15016,S+Niche,展厅,3
